In [1]:
import GN_ode as gn
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, odeint

# Test of Gauss-Newton method for ODE model
## Model
A homogeneous gas phase reaction
$$2NO+O_2\leftrightarrow2NO_2$$
is described by the following equation
$$\frac{dy}{dt}=k_1(126.2-y)(91.9-y)^2-k_2y^2;~~y(0)=0$$

In [2]:
def homo_gas(y,k):
    dydt = k[0]*(126.2-y)*(91.9-y)**2-k[1]*y**2
    return dydt

Pyrolytic dehydrogenation of dehzene to diphenyl and triphenyl
$$2C_2H_6\longleftrightarrow C_{12}H_{10}+H_2$$
$$C_6H_6+C_{12}H_{10}\longleftrightarrow C_{10}H_{14}+H_2$$
with differential equations model
\begin{align*}
\frac{dy_1}{dt}=&-r_1-r_2\\
\frac{dy_2}{dt}=&\frac{r_1}{2}-r_2\\
r_1=&k_1\left(y_1^2-y_2\frac{2-2y_1-y_2}{3K_1}\right)\\
r_2=&k_2\left(y_1y_2-\frac{(1-y_1-2y_2)(2-2y_1-y_2)}{9K_2}\right)
\end{align*}
with $K_1=0.242$ and $K_2=0.428$

In [3]:
def dehydro_ben(y,k):
    K1 = 0.242
    K2 = 0.428
    r1 = k[0]*(y[0]**2-y[1]*(2-2*y[0]-y[1])/(3*K1))
    r2 = k[1]*(y[0]*y[1]-(1-y[0]-2*y[1])*(2-2*y[0]-y[1])/(9*K2))
    dydt = np.empty(2)
    dydt[0] = -r1-r2
    dydt[1] = r1/2-r2
    return dydt

## Jacobian
Differential equation $\frac{d\mathbf{J}}{dt}$ with $\mathbf{J}=\frac{\partial\mathbf{y}}{\partial\mathbf{k}}$ is
$$\frac{d\mathbf{J}}{dt}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}$$
For this model
$$\frac{\partial f}{\partial y}=-k_1(91.9-y)^2-2k_1(126.2-y)(91.9-y)-2k_2y$$
$$\frac{\partial f}{\partial k_1}=(126.2-y)(91.9-y)^2$$
$$\frac{\partial f}{\partial k_2}=-y^2$$

In [4]:
def homo_gas_dfdy(y,k):
    dfdy = -k[0]*(91.9-y)**2-2*k[0]*(126.2-y)*(91.9-y)-2*k[1]*y
    return dfdy
def homo_gas_dfdk(y,k):
    dfdk = np.empty(2)
    dfdk[0] = (126.2-y)*(91.9-y)**2
    dfdk[1] = -y**2
    return dfdk

For the dehydrogenation model
\begin{align*}
r_1=&2y_1+\frac{2y_2}{3K_1}\\
r_2=&\frac{2y_1+2y_2-2}{3K_1}\\
r_3=&y_2-\frac{4y_1+5y_2-4}{9K_2}\\
r_4=&y_1-\frac{5y_1+4y_2-5}{9K_2}\\\\
\frac{\partial f_1}{\partial y_1}=&-k_1r_1-k_2r_3,\hspace{1cm}\frac{\partial f_1}{\partial y_2}=-k_1r_2-k_2r_4\\
\frac{\partial f_2}{\partial y_1}=&\frac{k_1}{2}r_1-k_2r_3,\hspace{1.3cm}\frac{\partial f_2}{\partial y_2}=\frac{k_1}{2}r_2-k_2r_4\\\\
\frac{\partial f_1}{\partial k_1}=&-\left(y_1^2+\frac{y_2^2+2y_1y_2-2y_2}{3K_1}\right)\\
\frac{\partial f_1}{\partial k_2}=&-\left(y_1y_2-\frac{2y_1^2-4y_1+5y_1y_2-5y_2+2y_2^2+2}{9K_2}\right)\\
\frac{\partial f_2}{\partial k_1}=&-\frac{1}{2}\frac{\partial f_1}{\partial k_1}\\
\frac{\partial f_2}{\partial k_2}=&\frac{\partial f_1}{\partial k_2}
\end{align*}

In [5]:
def dehydro_dfdy(y,k):
    K1 = 0.242
    K2 = 0.428
    r1 = 2*y[0]+2*y[1]/(3*K1)
    r2 = 2*(y[0]+y[1]-1)/(3*K1)
    r3 = y[1]-(4*y[0]+5*y[1]-4)/(9*K2)
    r4 = y[0]-(5*y[0]+4*y[1]-5)/(9*K2)
    dfdy = np.empty((2,2))
    dfdy[0,0] = -k[0]*r1-k[1]*r3
    dfdy[0,1] = -k[0]*r2-k[1]*r4
    dfdy[1,0] = k[0]*r1/2-k[1]*r3
    dfdy[1,1] = k[0]*r2/2-k[1]*r4
    return dfdy
def dehydro_dfdk(y,k):
    K1 = 0.242
    K2 = 0.428
    dfdk = np.empty((2,2))
    dfdk[0,0] = -(y[0]**2+(y[1]**2+2*y[0]*y[1]-2*y[1])/(3*K1))
    dfdk[0,1] = -(y[0]*y[1]-(2*y[0]**2-4*y[0]+5*y[0]*y[1]-5*y[1]+2*y[1]**2+2)/(9*K2))
    dfdk[1,0] = -dfdk[0,0]/2
    dfdk[1,1] = dfdk[0,1]
    return dfdk

## Comparison with numerical differentiation
Numerical differentiation by GN_ode functions and analytic results given above will be compared. Homogeneous gas model has 15 measurements and the parameter estimations are $k_1=0.4577\times10^{-5}$ and $k_2=0.2796\times10^{-3}$

In [6]:
yhat_homo=np.array([0,1.4,6.3,10.5,14.2,17.6,21.4,23.0,27.0,30.5,34.4,38.8,41.6,43.5,45.3])
t = np.array([0,1,2,3,4,5,6,7,9,11,14,19,24,29,39])
k = np.array([0.4577e-5,0.2796e-3])
Y = homo_gas(yhat_homo,k)
print(Y)

[4.87833129 4.67780141 4.01002815 3.47801019 3.0384812  2.65741801
 2.25603221 2.09441915 1.70858588 1.39121583 1.05831498 0.707007
 0.4958222  0.35762779 0.23031935]


In [7]:
dfdy_anal = np.empty(15)
dfdy_num = np.empty(15)
dfdk_anal = np.empty((2,15))
dfdk_num = np.empty((2,15))
for i in range(15):
    dfdy_anal[i] = homo_gas_dfdy(yhat_homo[i],k)
    dfdy_num[i] = gn.dfdy_ode(homo_gas,yhat_homo[i],k,1)
    dfdk_anal[:,i] = homo_gas_dfdk(yhat_homo[i],k)
    dfdk_num[:,i] = gn.dfdk_ode(homo_gas,yhat_homo[i],k,1,2)
print(dfdk_anal[0])
print(dfdk_anal[1])

[1065835.982 1022143.2    878550.464  766623.572  676176.48   599525.214
  520882.2    489912.072  417831.392  360785.172  303513.75   246433.914
  214045.614  193729.712  175679.204]
[-0.00000e+00 -1.96000e+00 -3.96900e+01 -1.10250e+02 -2.01640e+02
 -3.09760e+02 -4.57960e+02 -5.29000e+02 -7.29000e+02 -9.30250e+02
 -1.18336e+03 -1.50544e+03 -1.73056e+03 -1.89225e+03 -2.05209e+03]


In [8]:
kkk = np.array([100,100])
dfdy_anal = np.empty(15)
dfdy_num = np.empty(15)
dfdk_anal = np.empty((2,15))
dfdk_num = np.empty((2,15))
for i in range(15):
    dfdy_anal[i] = homo_gas_dfdy(yhat_homo[i],kkk)
    dfdy_num[i] = gn.dfdy_ode(homo_gas,yhat_homo[i],kkk,1)
    dfdk_anal[:,i] = homo_gas_dfdk(yhat_homo[i],kkk)
    dfdk_num[:,i] = gn.dfdk_ode(homo_gas,yhat_homo[i],kkk,1,2)
print(dfdk_anal)
print(dfdk_num)

[[ 1.06583598e+06  1.02214320e+06  8.78550464e+05  7.66623572e+05
   6.76176480e+05  5.99525214e+05  5.20882200e+05  4.89912072e+05
   4.17831392e+05  3.60785172e+05  3.03513750e+05  2.46433914e+05
   2.14045614e+05  1.93729712e+05  1.75679204e+05]
 [-0.00000000e+00 -1.96000000e+00 -3.96900000e+01 -1.10250000e+02
  -2.01640000e+02 -3.09760000e+02 -4.57960000e+02 -5.29000000e+02
  -7.29000000e+02 -9.30250000e+02 -1.18336000e+03 -1.50544000e+03
  -1.73056000e+03 -1.89225000e+03 -2.05209000e+03]]
[[ 1.06583536e+06  1.02214217e+06  8.78549367e+05  7.66623020e+05
   6.76175952e+05  5.99525124e+05  5.20881638e+05  4.89911810e+05
   4.17831168e+05  3.60785052e+05  3.03513557e+05  2.46433727e+05
   2.14045495e+05  1.93729624e+05  1.75679289e+05]
 [ 0.00000000e+00 -1.49011612e+00 -4.02331352e+01 -1.10268593e+02
  -2.01165676e+02 -3.09944153e+02 -4.58210707e+02 -5.28991222e+02
  -7.28666782e+02 -9.30577517e+02 -1.18352473e+03 -1.50538981e+03
  -1.73039734e+03 -1.89207494e+03 -2.05226243e+03]]


Dehydrogenation model has eight measurements and the parameter estimations are $k_1=354.61$ and $k_2=400.23$

In [9]:
t_deh = np.array([5.63,11.32,16.97,22.62,34.00,39.70,45.20,169.7])*1e-4
yhat_deh = np.array([[0.828,0.704,0.622,0.565,0.499,0.482,0.470,0.443],
                   [0.0737,0.1130,0.1322,0.1400,0.1468,0.1477,0.1477,0.1476]])
y0_deh = yhat_deh[:,0]

k_deh = np.array([354.61,400.23])
dfdy_deh_ana = np.empty((8,2,2))
dfdy_deh_num = np.empty((8,2,2))
dfdk_deh_ana = np.empty((8,2,2))
dfdk_deh_num = np.empty((8,2,2))
for i in range(8):
    dfdy_deh_ana[i] = dehydro_dfdy(yhat_deh[:,i],k_deh)
    dfdy_deh_num[i] = gn.dfdy_ode(dehydro_ben,yhat_deh[:,i],k_deh,2)
    dfdk_deh_ana[i] = dehydro_dfdk(yhat_deh[:,i],k_deh)
    dfdk_deh_num[i] = gn.dfdk_ode(dehydro_ben,yhat_deh[:,i],k_deh,2,2)
print(np.max(np.abs(dfdy_deh_ana-dfdy_deh_num)))
print(np.max(np.abs(dfdk_deh_ana-dfdk_deh_num)))

6.087531346565811e-06
1.1726620770188823e-06


In [21]:
k10000 = np.array([1000,1000])
dfdy_deh_ana = np.empty((8,2,2))
dfdy_deh_num = np.empty((8,2,2))
dfdk_deh_ana = np.empty((8,2,2))
dfdk_deh_num = np.empty((8,2,2))
for i in range(8):
    dfdy_deh_ana[i] = dehydro_dfdy(yhat_deh[:,i],k10000)
    dfdy_deh_num[i] = gn.dfdy_ode(dehydro_ben,yhat_deh[:,i],k10000,2)
    dfdk_deh_ana[i] = dehydro_dfdk(yhat_deh[:,i],k10000)
    dfdk_deh_num[i] = gn.dfdk_ode(dehydro_ben,yhat_deh[:,i],k10000,2,2)
print(dfdk_deh_ana)
print(dfdk_deh_num)

[[[-6.58144455e-01 -5.92973850e-02]
  [ 3.29072227e-01 -5.92973850e-02]]

 [[-4.21060904e-01 -7.08474309e-02]
  [ 2.10530452e-01 -7.08474309e-02]]

 [[-2.73293972e-01 -6.38318060e-02]
  [ 1.36646986e-01 -6.38318060e-02]]

 [[-1.78453650e-01 -4.97256490e-02]
  [ 8.92268251e-02 -4.97256490e-02]]

 [[-7.60762617e-02 -2.72073848e-02]
  [ 3.80381309e-02 -2.72073848e-02]]

 [[-5.16051157e-02 -1.98581757e-02]
  [ 2.58025579e-02 -1.98581757e-02]]

 [[-3.52984711e-02 -1.38568037e-02]
  [ 1.76492355e-02 -1.38568037e-02]]

 [[ 2.25710744e-04  2.94279958e-04]
  [-1.12855372e-04  2.94279958e-04]]]
[[[-6.58144472e-01 -5.92990546e-02]
  [ 3.29072236e-01 -5.92962124e-02]]

 [[-4.21059099e-01 -7.08467951e-02]
  [ 2.10528128e-01 -7.08467951e-02]]

 [[-2.73294631e-01 -6.38294750e-02]
  [ 1.36647316e-01 -6.38323172e-02]]

 [[-1.78454229e-01 -4.97252017e-02]
  [ 8.92264040e-02 -4.97255570e-02]]

 [[-7.60763896e-02 -2.72073919e-02]
  [ 3.80381948e-02 -2.72073919e-02]]

 [[-5.16052978e-02 -1.98582484e-02]
  

## Test of $\varphi(\mathbf{z})$
With an artificial initial condition $y_0=10$, $\mathbf{J}=\begin{bmatrix}1000&2000\end{bmatrix}$
$$\frac{dy}{dt}=4.577\times10^{-6}(126.2-10)(91.9-10)^2-2.796\times10^{-4}\times10^2=3.5395$$
$$\frac{\partial f}{\partial y}=-4.577\times10^{-6}(91.9-10)^2-2\times4.577\times10^{-6}(126.2-10)(91.9-10)-2\times2.796\times10^{-4}\times10=-0.1234$$
$$\frac{\partial f}{\partial k_1}=(126.2-10)(91.9-10)^2=779424$$
$$\frac{\partial f}{\partial k_2}=-10^2=-100$$
The differential equation for Jacobian is
$$\frac{d\mathbf{J}}{dt}=-0.1234\times\begin{bmatrix}1000 & 2000\end{bmatrix}+\begin{bmatrix}779424&-100\end{bmatrix}=\begin{bmatrix}779301&-346.8\end{bmatrix}$$

In [11]:
y_temp = 10
dydt_temp = homo_gas(y_temp,k)
dfdy_temp = homo_gas_dfdy(y_temp,k)
dfdk_temp = homo_gas_dfdk(10,k)
J_temp = np.array([1000,2000])
dJdt_temp = dfdy_temp*J_temp+dfdk_temp
print(dJdt_temp)
z = np.empty(3)
z[0] = y_temp
z[1:] = J_temp
dJdt_num = gn.phi_z(homo_gas,z,k,1,2)
print(dJdt_num[1:])

[ 7.79300873e+05 -3.46818670e+02]
[ 7.79300873e+05 -3.46818788e+02]


Artificial initial condition $y0=[5~~10]$ and $\mathbf{J}=\begin{bmatrix}
200&300\\
150&50\end{bmatrix}$
$$\frac{d\mathbf{J}}{dt}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}$$

In [12]:
y_deh_temp0 = np.array([5,10])
J_deh_temp = np.array([[200,300],[150,50]])
dfdy_deh_temp = dehydro_ben(y_deh_temp0,k_deh)
dfdy_deh_temp = dehydro_dfdy(y_deh_temp0,k_deh)
dfdk_deh_temp = dehydro_dfdk(y_deh_temp0,k_deh)
dJdt_deh_ana = np.matmul(dfdy_deh_temp,J_deh_temp)+dfdk_deh_temp
z_deh = np.empty(6)
z_deh[0:2] = y_deh_temp0
z_deh[2:] = J_deh_temp.transpose().flatten()
dzdt_deh = gn.phi_z(dehydro_ben,z_deh,k_deh,2,2)
dJdt_deh_num = dzdt_deh[2:].reshape(2,2).transpose()
print(dJdt_deh_ana)
print(dJdt_deh_num)

[[-3508740.85315775 -3610034.82730877]
 [ 3563353.91956953  3407433.3131871 ]]
[[-3508741.33291654 -3610035.15135963]
 [ 3563354.37318194  3407433.77309991]]


## Integration with estimated parameters

In [13]:
y0 = yhat_homo[0]
Y_homo,J_homo = gn.state_jacob_int(homo_gas,y0,k,t)
print(Y_homo)

[[ 0.          4.54927429  8.52410322 12.02631169 15.13385962 17.90748572
  20.39634002 22.63844317 26.50804658 29.71156267 33.56817887 38.19757805
  41.3294756  43.48731834 46.05863949]]


In [14]:
Y_deh,J_deh = gn.state_jacob_int(dehydro_ben,y0_deh,k_deh,t_deh)
print(Y_deh)

[[0.828      0.70520036 0.62134695 0.56423837 0.49894951 0.4810753
  0.46932139 0.44329922]
 [0.0737     0.11210656 0.13120882 0.1406277  0.14722938 0.14811396
  0.14841563 0.14773839]]


## Calculation of $\Delta k$ and $\chi^2$ with guessed parameters
$$k_{guess}=\begin{bmatrix}1\times10^{-6}&1\times10^{-6}\end{bmatrix}$$

In [15]:
k_guess = np.array([1e-6,1e-6])
Y_guess,J_guess = gn.state_jacob_int(homo_gas,y0,k_guess,t)
Q = np.eye(1)
dk_homo = gn.delta_k(J_guess,Q,yhat_homo,Y_guess,2,15)
chi_homo = gn.chi_squared(yhat_homo,Y_guess,Q,15)
print('delta k is ',dk_homo)
print('chi squared is ',chi_homo)

solve for scalar y
delta k is  [2.38247336e-06 2.58667199e-03]
chi squared is  4011.7439406240596


$$k_\text{guess}=[10000~~10000]$$

In [16]:
k_deh_guess = np.array([10000,10000])
Y_deh_guess,J_deh_guess = gn.state_jacob_int(dehydro_ben,y0_deh,k_deh_guess,t_deh)
Q_deh = np.eye(2)
dk_deh = gn.delta_k(J_deh_guess,Q_deh,yhat_deh,Y_deh_guess,2,8)
chi_deh = gn.chi_squared(yhat_deh,Y_deh_guess,Q_deh,8)
print('delta k is ',dk_deh)
print('chi squared is ',chi_deh)

solve for vector y
delta k is  [-37961798.47913542  -5459357.3779261 ]
chi squared is  0.12153270838997939


In [17]:
y_deh_temp0 = np.array([5,10])
J_deh_temp = np.array([[200,300],[150,50]])
dfdy_deh_temp = dehydro_ben(y_deh_temp0,k_deh_guess)
dfdy_deh_temp = dehydro_dfdy(y_deh_temp0,k_deh_guess)
dfdk_deh_temp = dehydro_dfdk(y_deh_temp0,k_deh_guess)
dJdt_deh_ana = np.matmul(dfdy_deh_temp,J_deh_temp)+dfdk_deh_temp
z_deh_guess = np.empty(6)
z_deh_guess[0:2] = y_deh_temp0
z_deh_guess[2:] = J_deh_temp.transpose().flatten()
dzdt_deh = gn.phi_z(dehydro_ben,z_deh_guess,k_deh_guess,2,2)
dJdt_deh_num = dzdt_deh[2:].reshape(2,2).transpose()
print(dJdt_deh_ana)
print(dJdt_deh_num)

[[-1.02815533e+08 -1.05238281e+08]
 [ 9.66063644e+07  9.26542806e+07]]
[[-1.02815545e+08 -1.05238290e+08]
 [ 9.66063779e+07  9.26542933e+07]]


In [18]:
AAA = np.zeros((2,2))
bbb = np.zeros(2)
print('yhat is ',np.shape(yhat_deh))
print(yhat_deh)
print('Y is ',np.shape(Y_deh_guess))
print(Y_deh_guess)
for i in range(8):
    JQ = np.matmul(np.transpose(J_deh_guess[i]),Q_deh)
    JQJ = np.matmul(JQ,J_deh_guess[i])
    AAA += JQJ
    JQg = np.dot(JQ,yhat_deh[:,i]-Y_deh_guess[:,i])
    bbb += JQg
print(AAA)
print(bbb)
dddkkk = np.linalg.solve(AAA,bbb)
print(dddkkk)

yhat is  (2, 8)
[[0.828  0.704  0.622  0.565  0.499  0.482  0.47   0.443 ]
 [0.0737 0.113  0.1322 0.14   0.1468 0.1477 0.1477 0.1476]]
Y is  (2, 8)
[[0.828      0.44331    0.44329291 0.44329313 0.44329307 0.44329308
  0.4432931  0.44329309]
 [0.0737     0.14775208 0.14773584 0.14773602 0.14773597 0.14773598
  0.14773599 0.14773598]]
[[3.81038019e-17 1.33041863e-18]
 [1.33041863e-18 3.58672345e-16]]
[-1.45375208e-09 -2.00862559e-09]
[-37961798.47913542  -5459357.3779261 ]


## Running a step for $k^{(2)}$ with bisection rule

In [19]:
k_2,yyy,jjj = gn.bisect(homo_gas,yhat_homo,Q,k_guess,t,10)
Y_guess2 = gn.state_only_int(homo_gas,y0,k_2,t)
chi22 = gn.chi_squared(yhat_homo,Y_guess2,Q,15)
print(k_2)
print(chi22)

solve for scalar y
[3.38247336e-06 2.58767199e-03]
1739.8785983389623


In [22]:
dk_deh = gn.delta_k(J_deh_guess,Q_deh,yhat_deh,Y_deh_guess,2,8)
k2_deh,Y_check,J_check = gn.bisect(dehydro_ben,yhat_deh,Q_deh,k_deh_guess,t_deh,10)
print(k2_deh)

solve for vector y
solve for vector y
[-64144.13765456   -662.80737876]
